In [1]:
import pandas as pd
import unicodedata
import geopandas as gpd
import geopy
from geopy.geocoders import Photon
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

In [3]:
raw_data = pd.read_csv("2019-10-01_sigesguarda_-_Base_de_Dados-limpo.csv")
display(raw_data.head(n=5))
display(raw_data.info())

,ATENDIMENTO_BAIRRO_NOME,NATUREZA1_DESCRICAO,SUBCATEGORIA1_DESCRICAO,OCORRENCIA_DATA,ORIGEM_CHAMADO_DESCRICAO,REGIONAL_FATO_NOME
0,cic,alarmes,disparo de alarme,2009-01-01 15:14:00,SIGA,cic
1,fazendinha,roubo,transeunte,2009-01-01 15:22:00,153,portão
2,uberaba,animais,cão solto em via pública,2009-01-01 15:59:00,156,cajuru
3,sítio cercado,animais,cão solto em via pública,2009-01-01 16:13:00,156,bairro novo
4,tatuquara,alarmes,disparo de alarme,2009-01-01 16:29:00,SIGA,pinheirinho


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255916 entries, 0 to 255915
Data columns (total 6 columns):
ATENDIMENTO_BAIRRO_NOME     255916 non-null object
NATUREZA1_DESCRICAO         255916 non-null object
SUBCATEGORIA1_DESCRICAO     255916 non-null object
OCORRENCIA_DATA             255916 non-null object
ORIGEM_CHAMADO_DESCRICAO    255916 non-null object
REGIONAL_FATO_NOME          255916 non-null object
dtypes: object(6)
memory usage: 11.7+ MB


None

In [4]:
raw_data['ADDRESS_FINAL'] = raw_data['ATENDIMENTO_BAIRRO_NOME'].astype(str) + ', Curitiba, Parana, Brazil'   
raw_data.head()

,ATENDIMENTO_BAIRRO_NOME,NATUREZA1_DESCRICAO,SUBCATEGORIA1_DESCRICAO,OCORRENCIA_DATA,ORIGEM_CHAMADO_DESCRICAO,REGIONAL_FATO_NOME,ADDRESS_FINAL
0,cic,alarmes,disparo de alarme,2009-01-01 15:14:00,SIGA,cic,"cic, Curitiba, Parana, Brazil"
1,fazendinha,roubo,transeunte,2009-01-01 15:22:00,153,portão,"fazendinha, Curitiba, Parana, Brazil"
2,uberaba,animais,cão solto em via pública,2009-01-01 15:59:00,156,cajuru,"uberaba, Curitiba, Parana, Brazil"
3,sítio cercado,animais,cão solto em via pública,2009-01-01 16:13:00,156,bairro novo,"sítio cercado, Curitiba, Parana, Brazil"
4,tatuquara,alarmes,disparo de alarme,2009-01-01 16:29:00,SIGA,pinheirinho,"tatuquara, Curitiba, Parana, Brazil"


In [5]:
raw_data_grouped = raw_data.groupby("ADDRESS_FINAL")
df_address =  pd.DataFrame(raw_data_grouped)
df_address.drop(1, axis=1, inplace=True)
df_address["location"] = 0
df_address

,0,location
0,"abranches, Curitiba, Parana, Brazil",0
1,"ahú, Curitiba, Parana, Brazil",0
2,"alto boqueirão, Curitiba, Parana, Brazil",0
3,"alto da glória, Curitiba, Parana, Brazil",0
4,"alto da rua xv, Curitiba, Parana, Brazil",0
...,...,...
126,"vila vila maria antonieta, Curitiba, Parana, B...",0
127,"vista alegre, Curitiba, Parana, Brazil",0
128,"xaxim, Curitiba, Parana, Brazil",0
129,"água verde, Curitiba, Parana, Brazil",0


In [6]:
from geopy.extra.rate_limiter import RateLimiter

locator = Photon(timeout=20)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1, max_retries=2)
df_address['location'] = df_address[0].apply(geocode)
df_address.head()

,0,location
0,"abranches, Curitiba, Parana, Brazil","(Curitiba, Paraná, Brazil, (-25.4295963, -49.2..."
1,"ahú, Curitiba, Parana, Brazil","(Curitiba, Paraná, Brazil, (-25.4295963, -49.2..."
2,"alto boqueirão, Curitiba, Parana, Brazil","(Córrego Alto Boqueirão, 81770-440, Curitiba, ..."
3,"alto da glória, Curitiba, Parana, Brazil","(Brazil Picante, 1099, Rua Mauá, 80030-150, Ru..."
4,"alto da rua xv, Curitiba, Parana, Brazil","(Alto da Rua XV, Curitiba, Paraná, Brazil, (-2..."


In [7]:
df_address['point'] = df_address['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df_address[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_address['point'].tolist(), index=df_address.index)
df_address

,0,location,point,latitude,longitude,altitude
0,"abranches, Curitiba, Parana, Brazil","(Curitiba, Paraná, Brazil, (-25.4295963, -49.2...","(-25.4295963, -49.2712724, 0.0)",-25.429596,-49.271272,0.0
1,"ahú, Curitiba, Parana, Brazil","(Curitiba, Paraná, Brazil, (-25.4295963, -49.2...","(-25.4295963, -49.2712724, 0.0)",-25.429596,-49.271272,0.0
2,"alto boqueirão, Curitiba, Parana, Brazil","(Córrego Alto Boqueirão, 81770-440, Curitiba, ...","(-25.5309106, -49.2290427, 0.0)",-25.530911,-49.229043,0.0
3,"alto da glória, Curitiba, Parana, Brazil","(Brazil Picante, 1099, Rua Mauá, 80030-150, Ru...","(-25.41651, -49.2646701, 0.0)",-25.416510,-49.264670,0.0
4,"alto da rua xv, Curitiba, Parana, Brazil","(Alto da Rua XV, Curitiba, Paraná, Brazil, (-2...","(-25.4280673, -49.2511966, 0.0)",-25.428067,-49.251197,0.0
...,...,...,...,...,...,...
126,"vila vila maria antonieta, Curitiba, Parana, B...","(Planta Vila Maria I, Curitiba, Paraná, Brazil...","(-25.4847227, -49.314875433880225, 0.0)",-25.484723,-49.314875,0.0
127,"vista alegre, Curitiba, Parana, Brazil","(Córrego Vista Alegre, 81320-390, Curitiba, Pa...","(-25.4675574, -49.3161516, 0.0)",-25.467557,-49.316152,0.0
128,"xaxim, Curitiba, Parana, Brazil","(Curitiba, Paraná, Brazil, (-25.4295963, -49.2...","(-25.4295963, -49.2712724, 0.0)",-25.429596,-49.271272,0.0
129,"água verde, Curitiba, Parana, Brazil","(Feira Livre do Água Verde, Rua Professor Braz...","(-25.4629852, -49.2910168, 0.0)",-25.462985,-49.291017,0.0


In [8]:
df_address.drop(["location", "point", "altitude"], axis=1, inplace=True)
df_address

,0,latitude,longitude
0,"abranches, Curitiba, Parana, Brazil",-25.429596,-49.271272
1,"ahú, Curitiba, Parana, Brazil",-25.429596,-49.271272
2,"alto boqueirão, Curitiba, Parana, Brazil",-25.530911,-49.229043
3,"alto da glória, Curitiba, Parana, Brazil",-25.416510,-49.264670
4,"alto da rua xv, Curitiba, Parana, Brazil",-25.428067,-49.251197
...,...,...,...
126,"vila vila maria antonieta, Curitiba, Parana, B...",-25.484723,-49.314875
127,"vista alegre, Curitiba, Parana, Brazil",-25.467557,-49.316152
128,"xaxim, Curitiba, Parana, Brazil",-25.429596,-49.271272
129,"água verde, Curitiba, Parana, Brazil",-25.462985,-49.291017


In [9]:
merged_inner = pd.merge(left=raw_data,right=df_address, left_on='ADDRESS_FINAL', right_on=0)
merged_inner.drop([0], axis=1, inplace=True)
merged_inner

,ATENDIMENTO_BAIRRO_NOME,NATUREZA1_DESCRICAO,SUBCATEGORIA1_DESCRICAO,OCORRENCIA_DATA,ORIGEM_CHAMADO_DESCRICAO,REGIONAL_FATO_NOME,ADDRESS_FINAL,latitude,longitude
0,cic,alarmes,disparo de alarme,2009-01-01 15:14:00,SIGA,cic,"cic, Curitiba, Parana, Brazil",-25.429596,-49.271272
1,cic,alarmes,disparo de alarme,2009-01-01 19:45:00,SIGA,cic,"cic, Curitiba, Parana, Brazil",-25.429596,-49.271272
2,cic,invasão,invasão de equipamento/patrimônio público,2009-01-01 20:45:00,190,cic,"cic, Curitiba, Parana, Brazil",-25.429596,-49.271272
3,cic,alagamento,não informado,2009-01-01 21:45:00,153,cic,"cic, Curitiba, Parana, Brazil",-25.429596,-49.271272
4,cic,alarmes,disparo de alarme,2009-01-01 23:46:00,SIGA,cic,"cic, Curitiba, Parana, Brazil",-25.429596,-49.271272
...,...,...,...,...,...,...,...,...,...
255911,águas belas,escolta,não informado,2019-08-31 14:00:00,OFÍCIO,região metropolitana,"águas belas, Curitiba, Parana, Brazil",-25.531555,-49.188815
255912,estados,violação de medida protetiva lei maria da penha,não informado,2019-05-26 15:09:00,IMEDIATA,região metropolitana,"estados, Curitiba, Parana, Brazil",-25.429596,-49.271272
255913,jr taisa,apoio,apoio ao gm,2019-07-24 22:15:00,AOS GMS,boa vista,"jr taisa, Curitiba, Parana, Brazil",-25.233046,-48.826578
255914,planta deodoro,apoio,apoio ao cidadão - socorro,2019-07-31 10:48:00,FONE,região metropolitana,"planta deodoro, Curitiba, Parana, Brazil",-25.544439,-49.166721


In [33]:
heatmap_address = merged_inner.groupby(['latitude','longitude']).size().reset_index(name='count')
heatmap_address

,latitude,longitude,count
0,-26.273300,-51.723011,1300
1,-26.270663,-52.678161,1
2,-26.235785,-52.686735,1
3,-26.100714,-49.804129,2
4,-25.844960,-53.814364,4461
...,...,...,...
59,-23.298675,-51.153829,514
60,-23.295716,-50.077896,1
61,-23.270722,-52.946942,1
62,-22.716607,-51.811996,4614


In [35]:
def generateBaseMap(default_location=[-25.4947402, -49.4298817], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map = generateBaseMap()
HeatMap(data=heatmap_address[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

base_map